In [ ]:
from google.colab import drive
drive.mount('/content/drive') #this works to conect the drive to the script itself being then able to acess files such as the data.zip

Mounted at /content/drive


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import librosa.display
import seaborn as sns
import tensorflow as tf
from sklearn.neural_network import MLPClassifier #for MLP classifier
import keras
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import * #for CNN and LSTM, * means import everything
from tensorflow.keras.optimizers import SGD

from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
import pandas as pd
import io
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

# Read the database of containing the features for each sample
df = pd.read_csv('/content/drive/MyDrive/DACO_2021_2022/featuresV2.csv')

print(df)

              0         1         2  ...       171       172     labels
0     -0.017110 -0.013606 -0.019834  ... -1.857551 -1.852664  surprised
1     -0.028183 -0.012709  0.017063  ... -8.222637 -7.808563       fear
2     -0.041308  0.034605  0.050918  ... -7.446040 -5.487325       calm
3     -0.056769  0.024256  0.057170  ... -7.833958 -6.122277    neutral
4      0.012351 -0.009679 -0.078754  ... -6.650182 -5.679175      happy
...         ...       ...       ...  ...       ...       ...        ...
12157 -0.007879  0.027866 -0.015370  ...  0.116544  6.037195    disgust
12158 -0.005548 -0.006217  0.008452  ... -1.775892 -1.137908        sad
12159 -0.006007  0.013230 -0.036165  ... -3.349862 -3.459580      angry
12160  0.014099  0.009986 -0.056546  ...  2.672708  4.628762    disgust
12161 -0.028532  0.006661  0.030099  ... -3.585440 -4.979897  surprised

[12162 rows x 174 columns]


Normalize data with StandardScaler()

In [ ]:
Y = df.labels
size =  df.shape[1]-1
X = df.iloc[:, 0:size]
X = StandardScaler().fit_transform(X)

**Figure the best SVM model **

Split the Dataset (60% for training and 40% for validation)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X,Y , train_size=0.6, shuffle = True, random_state = 10)


print(X_train.shape) 
print(y_train.shape)
print(X_valid.shape) 
print(y_valid.shape)


(7297, 173)
(7297,)
(4865, 173)
(4865,)


We test *Principal Analysis Components*(PCA) as a pré-processing method for 
reduction of the dimensionality

In [ ]:
from sklearn.decomposition import PCA
# Make an instance of the Model
pca = PCA(.95) #It means that scikit-learn choose the minimum number of principal components such that 95% of the variance is retained.

pca.fit(X_train)

PCA(n_components=0.95)

In [ ]:
x_train_pca = pca.transform(X_train)
x_valid_pca = pca.transform(X_valid)

SVM poly kernel with and without PCA

In [ ]:
#Parameters tested in the loop using the validation dataset for "testing" and figure the best parameters

poly_degree = [1,2,3,4,5] 
C_1 = [4 ,7, 8, 9 , 10 , 15 , 20 ,25, 30, 100]

parameters_svm = {} #dict to save parameters info and scores
parameters_svm_pca = {} #dict to save parameters info and scores

for Degree in poly_degree: # loop for testing each value for each parameter
    for c in C_1:
        svm_model = SVC(kernel='poly',  degree= Degree, C = c )
        svm_model_pca = SVC(kernel='poly',  degree= Degree, C = c )

        svm_model.fit(X_train, y_train)
        svm_model_pca.fit(x_train_pca, y_train)
        
        score = svm_model.score(X_valid, y_valid) 
        score_pca= svm_model_pca.score(x_valid_pca, y_valid)

        parameters_svm.update( {(Degree, c): score})
        parameters_svm_pca.update( {(Degree, c): score_pca})


In [ ]:
best_parameters = max(parameters_svm, key=parameters_svm.get)
degree_opt = best_parameters[0]
c_opt = best_parameters[1]
print('Best parameters for degree = {} and C = {} and score {} '.format(degree_opt, c_opt,parameters_svm[degree_opt,c_opt]))


Best parameters for degree = 3 and C = 100 and score 0.6073997944501541 


In [ ]:
best_parameters_pca = max(parameters_svm_pca, key=parameters_svm_pca.get)
degree_opt = best_parameters_pca[0]
c_opt = best_parameters_pca[1]
print('Best parameters for degree = {} and C = {} and score with PCA {} '.format(degree_opt, c_opt,parameters_svm_pca[degree_opt,c_opt]))

Best parameters for degree = 3 and C = 100 and score with PCA 0.5971223021582733 


SVM kernel RBF(default) with and without PCA 

In [ ]:
C_1 = [4 ,7, 8, 9 , 10 , 15 , 20 ,25, 30, 100] # parameters to test for each iteration
parameters_svm = {} #dict to save parameters info and scores
parameters_svm_pca = {}

for c in C_1:
    svm_model = SVC( C = c ) #Regularization parameter. The strength of the regularization is inversely proportional to C. Must be strictly positive. The penalty is a squared l2 penalty.
    svm_model_pca = SVC( C = c )

    svm_model.fit(X_train, y_train)
    svm_model_pca.fit(x_train_pca, y_train)

    score = svm_model.score(X_valid, y_valid) #using the validation dataset for "testing" and figure the best parameters
    score_pca=svm_model_pca.score(x_valid_pca,y_valid)
    
    parameters_svm.update({c: score})
    parameters_svm_pca.update({c: score_pca})


In [ ]:
best_parameters = max(parameters_svm, key=parameters_svm.get)

print('Best parameters for C = {} and score = {} '.format(best_parameters,parameters_svm[best_parameters]))


Best parameters for C = 30 and score = 0.6357656731757452 


In [ ]:
best_parameters_pca = max(parameters_svm_pca, key=parameters_svm_pca.get)

print('Best parameters for C = {} and score = {} com PCA'.format(best_parameters_pca,parameters_svm_pca[best_parameters_pca]))


Best parameters for C = 15 and score = 0.6261048304213772 com PCA


WE FOUND THE BEST MODEL (SVM WITH RBF KERNEL C=30 , WITHOUT PCA)

NOW concatenate the training set and validation set , to train each fold in K-stratified validation

In [ ]:

svm_model = SVC( C = best_parameters)
x_final=np.concatenate((X_train,X_valid),axis=0)
y_final=np.concatenate((y_train,y_valid),axis=0)


**MODEL ASSESSMENT** --
Stratified - cross validation --This cross-validation object is a variation of KFold that returns stratified folds. The folds are made by preserving the percentage of samples for each class.

In [ ]:
from sklearn.model_selection import StratifiedKFold
scores_stratified = [] # list to append the scores of each of the 5 folds

report_metrics=[]# list to append each classification report of each fold

skf =StratifiedKFold(n_splits=5) #we choose 5 folds

for train_index, test_index in skf.split(x_final,y_final):
    #print("Train Index: ", train_index, "\n")
    #print("Test Index: ", test_index)
    
    X_train_cv, X_test_cv, y_train_cv, y_test_cv = x_final[train_index], x_final[test_index], y_final[train_index], y_final[test_index]
    svm_model.fit(X_train_cv, y_train_cv)
    
    scores_stratified.append(svm_model.score(X_test_cv, y_test_cv))
    y_pred=svm_model.predict(X_test_cv)
    report=classification_report(y_test_cv, y_pred,output_dict=True)
    df=pd.DataFrame(report).transpose()
    report_metrics.append(df)


In [ ]:
print('With stratified validation we get a score of {} (+/-) {} '.format(np.mean(scores_stratified),np.std(scores_stratified)))

With stratified validation we get a score of 0.6530184605317239 (+/-) 0.006169823843020579 


In [ ]:
report_metrics
kFoldResults_SVM = pd.concat([report_metrics[0], report_metrics[1], report_metrics[2], report_metrics[3], report_metrics[4]])
kFoldResults_SVM.to_csv('5kFoldResults_SVM_sem_augmentation.csv', index=True)